# 在任务中不同节点间传输数据

数据传输操作是指在一个任务的上下文环境之内，在不同节点之间传输数据。如果不在一个具体计算任务的环境中，数据传输工具不会工作。数据传输工具适合传递整份的大数据，如果用来传递零散的小数据，功能上虽然支持，但是性能达不到最优的状态。数据传输工具既支持 1->1 的传输，也支持 1->N 的传输。

## 数据通道接口简介

目前数据通道定义了四个接口，分别对应 1->1 发送、1->1 接收、1->N 发送、N->1 接收四种场景。

In [ ]:
from typing import Callable, Dict, List, Tuple
from alphafed.contractor.common import ContractEvent

In [ ]:
def send_stream(self,
                source: str,
                target: str,
                data_stream: bytes,
                connection_timeout: int = 30,
                timeout: int = 60,
                **kwargs) -> str | None:
    """1->1 发送数据流。
    
    参数说明:
        source:
            发送方 ID。
        target:
            接收方 ID。
        data_stream:
            数据流对象。
        connection_timeout:
            连接超时时间。
        timeout:
            传输超时时间。
        kwargs:
            其它扩展参数，由实现接口的具体实现类定义。

    返回值:
        发送成功返回接收方节点 ID，否则返回 None。
    """

def batch_send_stream(self,
                      source: str,
                      target: List[str],
                      data_stream: bytes,
                      connection_timeout: int = 30,
                      timeout: int = 60,
                      ensure_all_succ: bool = False,
                      **kwargs) -> List[str] | None:
    """1->N 发送数据流。
    
    参数说明:
        source:
            发送方 ID。
        target:
            接收方 ID。
        data_stream:
            数据流对象。
        connection_timeout:
            连接超时时间。
        timeout:
            传输超时时间。
        ensure_all_succ: 
            是否要求确保发送全部成功？如果设置为 True，任何一个接收方接收失败都会触发异常。
            否则返回接收成功的节点 ID 列表，忽略接收失败的情况。
        kwargs: 
            其它扩展参数，由实现接口的具体实现类定义。

    返回值:
        返回接收成功的节点 ID 列表。
    """

def receive_stream(self,
                   receiver: str,
                   source: str,
                   complementary_handler: Callable[[ContractEvent], None] = None,
                   timeout: int = 0,
                   **kwargs) -> Tuple[str, bytes] | None:
    """1->1 接收数据流。
    
    参数说明:
        receiver:
            数据接收方 ID。
        source:
            出于安全考虑，接收数据时必须指定数据源节点的 ID，其它节点的发送请求会被拒绝。
        complementary_handler:
            接收数据期间可能会继续收到与数据传输无关的其它合约消息，如果想要处理这些消息，
            可以通过 complementary_handler 设置自定义的处理函数。当收到任意一个非
            数据传输消息时，会通过调用 complementary_handler(event_obj) 对其进行
            处理。
        timeout:
            接收数据超时时间。
        kwargs:
            其它扩展参数，由实现接口的具体实现类定义。

    返回值:
        “发送者节点 ID”，“收到的 bytes 格式数据流”组成的元组。
    """

def batch_receive_stream(self,
                         receiver: str,
                         source_list: List[str],
                         complementary_handler: Callable[[ContractEvent], None] = None,
                         timeout: int = 0,
                         ensure_all_succ: bool = False,
                         **kwargs) -> Dict[str, bytes] | None:
    """N->1 接收数据流。
    
    参数说明:
        receiver:
            数据接收方 ID。
        source_list:
            出于安全考虑，接收数据时必须指定数据源节点的 ID 列表，只有列表中指定节点
            发送的数据会被接收，其它节点发送的传输请求会被拒绝。
        complementary_handler:
            接收数据期间可能会继续收到与数据传输无关的其它合约消息，如果想要处理这些消息，
            可以通过 complementary_handler 设置自定义的处理函数。当收到任意一个非
            数据传输消息时，会通过调用 complementary_handler(event_obj) 对其进行
            处理。
        timeout:
            接收数据超时时间。
        ensure_all_succ:
            是否要求确保接收全部成功？如果设置为 True，从任何一个发送方接收数据失败都会
            触发异常。否则返回接收成功的节点 ID 和数据，忽略接收失败的情况。
        kwargs:
            其它扩展参数，由实现接口的具体实现类定义。

    返回值:
        接收成功的信息字典，key 为发送者节点 ID，value 为收到的 bytes 格式数据流。
    """

## 数据通道工具

AlphaMed 平台提供了一个实现了上述全部接口的数据传输工具 `SharedFileDataChannel`。数据传输时必须以 bytes 字节流的形式传输，以兼容不同的原始数据类型，并避免在传输过程中损坏数据格式。

发送数据时需要先实例化一个 `SharedFileDataChannel` 对象，实例化对象时需要传入当前任务使用的合约消息工具对象。因为数据传输的过程需要参与节点之间协调流程，因此需要发送、接收任务级合约消息。

In [ ]:
from alphafed.contractor import TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

contractor = TaskMessageContractor(task_id='TASK_ID')
data_channel = SharedFileDataChannel(contractor=contractor)
data_stream = 'Some data to transfer'.encode()
# 发送给单个节点
received = data_channel.send_stream(source='Self_ID',
                                    target='Target_ID',
                                    data_stream=data_stream)
# 发送给一批节点
received = data_channel.batch_send_stream(source='Self_ID',
                                          target=['Target_ID_1', 'Target_ID_2', ...],
                                          data_stream=data_stream,
                                          ensure_all_succ=True)

接收数据时也需要先实例化 `SharedFileDataChannel` 对象，方法与发送数据时相同。

In [ ]:
from alphafed.contractor import TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

contractor = TaskMessageContractor(task_id='TASK_ID')
data_channel = SharedFileDataChannel(contractor=contractor)

event_cache = []
def event_handler(event: ContractEvent):
    """记录传输过程中收到的事件，传输完成后继续处理。"""
    event_cache.append(event)

# 从单个节点接收数据
sender, data = data_channel.receive_stream(receiver='Self_ID',
                                           source='Source_ID',
                                           complementary_handler=event_handler)
# 从多个节点批量接收数据
data_dict = data_channel.batch_receive_stream(receiver='Self_ID',
                                              source_list=['Source_ID_1', 'Source_ID_2', ...],
                                              complementary_handler=event_handler,
                                              ensure_all_succ=True)
for _sender, _data in data_dict.items():
    # 逐个处理收到的数据
    ...

# 重放传输期间收到的事件并进行处理
for _event in event_cache:
    ...

## 使用数据通道工具传输数据实例

以下实例演示在模拟环境中使用数据通道工具，从一个节点向另外两个节点传输一条文本信息。**注意这里只是为了演示，实际使用中一个字符串这样的小数据不适合使用数据通道工具传输，而是建议使用合约消息进行传递。**

In [ ]:
# 发送方脚本
from alphafed import mock_context
from alphafed.contractor import TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

task_id = 'b2f615fb-f2e7-4aa0-b5fb-a4fd68c6f38a'  # 指定一个假想 ID
sender_id = 'ca28a832-cc13-40a3-8292-dee4c960a6cb'  # 指定一个假想 ID
receiver_id_1 = '98cf65a2-53dc-4ee4-b261-7cee17f42355'  # 指定一个假想 ID
receiver_id_2 = 'ff433f39-93cf-4bc8-8040-d4392a6fd139'  # 指定一个假想 ID
receiver_ids = [receiver_id_1, receiver_id_2]
contractor = TaskMessageContractor(task_id=task_id)
data_channel = SharedFileDataChannel(contractor=contractor)
data_stream = 'A message from sender.'.encode()  # 数据要转为 bytes

print('准备开始传输数据')
with mock_context(id=sender_id, nodes=[sender_id, *receiver_ids]):
    print('使用一对一的方式轮流传输 ...')
    for _receiver in receiver_ids:
        received = data_channel.send_stream(source=sender_id,
                                            target=_receiver,
                                            data_stream=data_stream)
    print(f'数据传输完成, 节点 {received} 收到了数据')
    print('使用一对多的方式批量传输 ...')
    received = data_channel.batch_send_stream(source=sender_id,
                                              target=receiver_ids,
                                              data_stream=data_stream,
                                              ensure_all_succ=True)
    print(f'数据传输完成, 节点 {received} 收到了数据')

    print('与接收方 1 一起向接收方 2 发送消息。')
    received = data_channel.send_stream(source=sender_id,
                                        target=receiver_id_2,
                                        data_stream=data_stream)
    print(f'数据传输完成, 节点 {received} 收到了数据')

In [ ]:
# 接收方 1 脚本
from alphafed import mock_context
from alphafed.contractor import ContractEvent, TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

# 以下 ID 要与发送方设置相同
task_id = 'b2f615fb-f2e7-4aa0-b5fb-a4fd68c6f38a'
sender_id = 'ca28a832-cc13-40a3-8292-dee4c960a6cb'
receiver_id_1 = '98cf65a2-53dc-4ee4-b261-7cee17f42355'
receiver_id_2 = 'ff433f39-93cf-4bc8-8040-d4392a6fd139'
receiver_ids = [receiver_id_1, receiver_id_2]
contractor = TaskMessageContractor(task_id=task_id)
data_channel = SharedFileDataChannel(contractor=contractor)

def complementary_handler(event: ContractEvent):
    # 处理其它事件
    pass

print('准备开始接收数据')
with mock_context(id=receiver_id_1, nodes=[sender_id, *receiver_ids]):
    print('发送方一对一发送数据')
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_1,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')
    print('发送方一对多发送数据')
    # 接收方的接收方式是一样的
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_1,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')

    print('与发送方一起向接收方 2 发送消息。')
    data_stream = 'A message from sender.'.encode()  # 数据要转为 bytes
    received = data_channel.send_stream(source=receiver_id_1,
                                        target=receiver_id_2,
                                        data_stream=data_stream)
    print(f'数据传输完成, 节点 {received} 收到了数据')

In [ ]:
# 接收方 2 脚本
from alphafed import mock_context
from alphafed.contractor import ContractEvent, TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

# 以下 ID 要与发送方设置相同
task_id = 'b2f615fb-f2e7-4aa0-b5fb-a4fd68c6f38a'
sender_id = 'ca28a832-cc13-40a3-8292-dee4c960a6cb'
receiver_id_1 = '98cf65a2-53dc-4ee4-b261-7cee17f42355'
receiver_id_2 = 'ff433f39-93cf-4bc8-8040-d4392a6fd139'
receiver_ids = [receiver_id_1, receiver_id_2]
contractor = TaskMessageContractor(task_id=task_id)
data_channel = SharedFileDataChannel(contractor=contractor)

def complementary_handler(event: ContractEvent):
    # 处理其它事件
    pass

print('准备开始接收数据')
with mock_context(id=receiver_id_2, nodes=[sender_id, *receiver_ids]):
    print('发送方一对一发送数据')
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_2,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')
    print('发送方一对多发送数据')
    # 接收方的接收方式是一样的
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_2,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')

    print('发送方与接收方 1 一起向我发送消息。')
    data_dict = data_channel.batch_receive_stream(receiver=receiver_id_2,
                                                  source_list=[sender_id, receiver_id_1],
                                                  complementary_handler=complementary_handler,
                                                  ensure_all_succ=True)
    for _source, _data_stream in data_dict.items():
        print(f'接收到 {_source} 发送的消息：`{_data_stream.decode()}`')

两个接收方的脚步代码相同，但是需要各自使用不同的 ID。和以前一样将脚本放置在三个不同的 .ipynb 文件中运行，模拟三个参与方节点。也可以直接运行[传输数据发送方](res/7_sender.ipynb)、[传输数据接收方 1](res/7_receiver_1.ipynb)、[传输数据接收方 2](res/7_receiver_2.ipynb)三个脚本，体验数据传输效果。